In [ ]:
%pip install PyMuPDF
%pip install PyPDF2
%pip install faiss-gpu
# pip install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 929.7 kB/s eta 0:00:00eta 0:00:010:00:030m
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from typing import List, Dict
from pathlib import Path
import fitz  # PyMuPDF
from dotenv import load_dotenv
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
# from langchain_community.llms import Together
from langchain_together import Together as TogetherLLM
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [2]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"

In [3]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

/tmp/ipykernel_278124/3353852039.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)


In [5]:
import getpass

TOGETHER_API_KEY = getpass.getpass("Get your TogetherAPI key ")

In [6]:

# class TogetherEmbedding:
#     def __init__(self, api_key: str = None, model: str = "BAAI/bge-large-en-v1.5"):
#         # Use the API key from the parameter or the environment.
#         self.client = Together(api_key=api_key or os.environ.get("TOGETHER_API_KEY"))
#         self.model_api_string = model

#     def get_text_embeddings(self, input_texts: List[str]) -> List[List[float]]:
#         outputs = self.client.embeddings.create(
#             input=input_texts,
#             model=self.model_api_string,
#         )
#         # Each output in outputs.data should have an 'embedding' attribute.
#         return np.array([x.embedding for x in outputs.data]).tolist()

#     def get_text_embedding(self, text: str) -> List[float]:
#         # Wrap a single text in a list and return its embedding.
#         return self.get_text_embeddings([text])[0]

#     def get_query_embedding(self, query: str) -> List[float]:
#         return self.get_text_embedding(query)

In [7]:
# Initialize the TogetherLLM using the official integration.
llm = TogetherLLM(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    api_key=TOGETHER_API_KEY,
    # model="Qwen/Qwen2-72B-Instruct",  # or any model of your choice
    # api_key = os.environ.get("TOGETHER_API_KEY"),  # Replace with your actual API key or set it in your environment
    max_tokens=512,
    # temperature=0.0,
)

In [8]:
vector_store = None
conversation_chain = None

In [9]:
memory = ConversationBufferMemory(
    return_messages=True,
    output_key="answer",
    input_key="question"
)

/tmp/ipykernel_278124/2898383963.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [20]:
# memory = ConversationBufferMemory(
#     memory_key="chat_history",
#     return_messages=True
# )

/tmp/ipykernel_249227/515166408.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [10]:
def process_pdf(pdf_path: str) -> str:
    """Extract text from PDF file using PyMuPDF."""
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

In [11]:
def process_documents(file_paths: List[str]) -> None:
        """Process multiple documents and create vector store."""
        documents = []
        
        for file_path in file_paths:
            if file_path.endswith('.pdf'):
                text = process_pdf(file_path)
                doc = Document(
                    page_content=text,
                    metadata={"source": file_path}
                )
                documents.append(doc)
        
        # Split documents into chunks
        splits = text_splitter.split_documents(documents)
        
        # Create or update vector store
        if vector_store is None:
            vector_store = FAISS.from_documents(splits, embeddings)
        else:
            vector_store.add_documents(splits)
        
        # Initialize conversation chain
        conversation_chain = ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=vector_store.as_retriever(
                search_kwargs={"k": 3}
            ),
            memory=memory,
            return_source_documents=True,
        )

In [12]:
def query(question: str) -> Dict:
        """Query the RAG system with a question."""
        if conversation_chain is None:
            raise ValueError("Please process documents before querying.")
        
        response = conversation_chain({"question": question})
        
        return {
            "answer": response["answer"],
            "source_documents": [
                {
                    "content": doc.page_content,
                    "source": doc.metadata["source"]
                }
                for doc in response["source_documents"]
            ]
        }

In [13]:
def save_vector_store(path: str) -> None:
        """Save the vector store to disk."""
        if vector_store is not None:
            vector_store.save_local(path)

In [14]:
def load_vector_store(path: str) -> None:
        """Load the vector store from disk."""
        if os.path.exists(path):
            vector_store = FAISS.load_local(path, embeddings)
            conversation_chain = ConversationalRetrievalChain.from_llm(
                llm=llm,
                retriever=vector_store.as_retriever(
                    search_kwargs={"k": 3}
                ),
                memory=memory,
                return_source_documents=True,
            ) 

In [15]:
import os

    
# Path to your PDF documents
pdf_directory = "/home/teng/development/projects/public/efh-hackathon/notebooks/data/Agroforestry"
pdf_files = [
    os.path.join(pdf_directory, f) 
    for f in os.listdir(pdf_directory) if f.endswith('.pdf')
]
    
# Process the documents
print("Processing documents...")
process_documents(pdf_files)
    
# Save the vector store for future use
save_vector_store("vector_store")
    

Processing documents...


UnboundLocalError: local variable 'vector_store' referenced before assignment

In [16]:
# import os
# from typing import List, Dict
# from pathlib import Path
# import fitz  # PyMuPDF
# from dotenv import load_dotenv
# from langchain_core.documents import Document
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_community.vectorstores import FAISS
# # from langchain_community.llms import Together
# from langchain_together import Together as TogetherLLM
# from langchain.chains import ConversationalRetrievalChain
# from langchain.memory import ConversationBufferMemory

# class RAGAgent:
#     def __init__(self, model_name: str = "mistralai/Mixtral-8x7B-Instruct-v0.1"):
#         load_dotenv()
        
#         # Initialize embeddings
#         self.embeddings = HuggingFaceEmbeddings(
#             model_name="sentence-transformers/all-mpnet-base-v2"
#         )
        
#         # Initialize text splitter
#         self.text_splitter = RecursiveCharacterTextSplitter(
#             chunk_size=1000,
#             chunk_overlap=200,
#             length_function=len,
#         )
        
#         # Initialize LLM
#         self.llm = TogetherLLM(
#             api_key=TOGETHER_API_KEY,
#             model=model_name,
#             temperature=0.7,
#             max_tokens=2048,
#         )
        
#         self.vector_store = None
#         self.conversation_chain = None
#         self.memory = ConversationBufferMemory(
#             return_messages=True,
#             output_key="answer",
#             input_key="question"
#         )

#     def process_pdf(self, pdf_path: str) -> str:
#         """Extract text from PDF file using PyMuPDF."""
#         text = ""
#         with fitz.open(pdf_path) as doc:
#             for page in doc:
#                 text += page.get_text()
#         return text

#     def process_documents(self, file_paths: List[str]) -> None:
#         """Process multiple documents and create vector store."""
#         documents = []
        
#         for file_path in file_paths:
#             if file_path.endswith('.pdf'):
#                 text = self.process_pdf(file_path)
#                 doc = Document(
#                     page_content=text,
#                     metadata={"source": file_path}
#                 )
#                 documents.append(doc)
        
#         # Split documents into chunks
#         splits = self.text_splitter.split_documents(documents)
        
#         # Create or update vector store
#         if self.vector_store is None:
#             self.vector_store = FAISS.from_documents(splits, self.embeddings)
#         else:
#             self.vector_store.add_documents(splits)
        
#         # Initialize conversation chain
#         self.conversation_chain = ConversationalRetrievalChain.from_llm(
#             llm=self.llm,
#             retriever=self.vector_store.as_retriever(
#                 search_kwargs={"k": 3}
#             ),
#             memory=self.memory,
#             return_source_documents=True,
#         )

#     def query(self, question: str) -> Dict:
#         """Query the RAG system with a question."""
#         if self.conversation_chain is None:
#             raise ValueError("Please process documents before querying.")
        
#         response = self.conversation_chain({"question": question})
        
#         return {
#             "answer": response["answer"],
#             "source_documents": [
#                 {
#                     "content": doc.page_content,
#                     "source": doc.metadata["source"]
#                 }
#                 for doc in response["source_documents"]
#             ]
#         }

#     def save_vector_store(self, path: str) -> None:
#         """Save the vector store to disk."""
#         if self.vector_store is not None:
#             self.vector_store.save_local(path)

#     def load_vector_store(self, path: str) -> None:
#         """Load the vector store from disk."""
#         if os.path.exists(path):
#             self.vector_store = FAISS.load_local(path, self.embeddings)
#             self.conversation_chain = ConversationalRetrievalChain.from_llm(
#                 llm=self.llm,
#                 retriever=self.vector_store.as_retriever(
#                     search_kwargs={"k": 3}
#                 ),
#                 memory=self.memory,
#                 return_source_documents=True,
#             ) 

In [33]:
import os
import warnings
from typing import List, Dict
from pathlib import Path
import fitz  # PyMuPDF
from dotenv import load_dotenv
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
# from langchain_community.llms import Together
from langchain_together import Together as TogetherLLM
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# Disable tokenizers parallelism warning
os.environ["TOKENIZERS_PARALLELISM"] = "false"

class RAGAgent:
    def __init__(self, model_name: str = "mistralai/Mixtral-8x7B-Instruct-v0.1"):
        load_dotenv()
        
        # Initialize embeddings
        self.embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-mpnet-base-v2"
        )
        
        # Initialize text splitter
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len,
        )
        
        # Initialize LLM
        self.llm = TogetherLLM(
            api_key=TOGETHER_API_KEY,
            model=model_name,
            temperature=0.7,
            max_tokens=2048,
        )
        
        self.vector_store = None
        self.conversation_chain = None
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            output_key="answer",
            input_key="question",
            return_messages=True
        )

    def process_pdf(self, pdf_path: str) -> str:
        """Extract text from PDF file using PyMuPDF."""
        text = ""
        with fitz.open(pdf_path) as doc:
            for page in doc:
                text += page.get_text()
        return text

    def process_documents(self, file_paths: List[str]) -> None:
        """Process multiple documents and create vector store."""
        documents = []
        
        for file_path in file_paths:
            if file_path.endswith('.pdf'):
                text = self.process_pdf(file_path)
                doc = Document(
                    page_content=text,
                    metadata={"source": file_path}
                )
                documents.append(doc)
        
        # Split documents into chunks
        splits = self.text_splitter.split_documents(documents)
        
        # Create or update vector store
        if self.vector_store is None:
            self.vector_store = FAISS.from_documents(splits, self.embeddings)
        else:
            self.vector_store.add_documents(splits)
        
        # Initialize conversation chain
        self.conversation_chain = ConversationalRetrievalChain.from_llm(
            llm=self.llm,
            retriever=self.vector_store.as_retriever(
                search_kwargs={"k": 3}
            ),
            memory=self.memory,
            return_source_documents=True,
            verbose=True,
            return_generated_question=False,
            chain_type="stuff",
            rephrase_question=False
        )

    def query(self, question: str) -> Dict:
        """Query the RAG system with a question."""
        if self.conversation_chain is None:
            raise ValueError("Please process documents before querying.")
        
        try:
            response = self.conversation_chain({"question": question})
            return {
                "answer": response["answer"],
                "source_documents": [
                    {
                        "content": doc.page_content,
                        "source": doc.metadata["source"]
                    }
                    for doc in response["source_documents"]
                ]
            }
        except Exception as e:
            print(f"Error during query: {str(e)}")
            return {
                "answer": "Sorry, I encountered an error while processing your question.",
                "source_documents": []
            }

    def save_vector_store(self, path: str) -> None:
        """Save the vector store to disk."""
        if self.vector_store is not None:
            self.vector_store.save_local(path)

    def load_vector_store(self, path: str) -> None:
        """Load the vector store from disk."""
        if os.path.exists(path):
            self.vector_store = FAISS.load_local(path, self.embeddings)
            self.conversation_chain = ConversationalRetrievalChain.from_llm(
                llm=self.llm,
                retriever=self.vector_store.as_retriever(
                    search_kwargs={"k": 3}
                ),
                memory=self.memory,
                return_source_documents=True,
            ) 

In [34]:
import os

# Initialize the RAG agent
agent = RAGAgent()

# Path to your PDF documents
pdf_directory = "/home/teng/development/projects/public/efh-hackathon/notebooks/data/Agroforestry"
# projects/public/efh-hackathon/notebooks/data
# projects/public/efh-hackathon/notebooks/data/Agroforestry
pdf_files = [
    os.path.join(pdf_directory, f) 
    for f in os.listdir(pdf_directory)
    if f.endswith('.pdf')
]

# Process the documents
print("Processing documents...")
agent.process_documents(pdf_files)

# Save the vector store for future use
agent.save_vector_store("vector_store")

# Test the RAG system
questions = [
    "How do different tree intervention practices impact total income?"
]

for question in questions:
    print(f"\nQuestion: {question}")
    response = agent.query(question)
    print("\nAnswer:", response["answer"])
    print("\nSources:")
    for doc in response["source_documents"]:
        print(f"  Source: {doc['source']}")
        print(f"  Relevant content: {doc['content'][:200]}...")

Processing documents...

Question: How do different tree intervention practices impact total income?


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

timber production and non-timber forest products. The 
studies suggest that tree-based interventions could 
negatively affect income from timber production, 
although the pooled effects were from only three studies 
and were non-significant at 95% CI (−0·13 [−0·29 to 0·02]; 
p=0·093; I²=37·5% and τ²=0·01; figure 4B). It is probable 
that income from non-timber forest products could 
address, to some extent, the deficit generated by reduced 
timber-related revenue, as suggested by the effects pooled 
across seven studies (11 analyses) that reported on this 
outcome (0·32 [0·04 to 0·61]; p=0·026; I²=94·1%, 
τ²=0·14; figure 

In [35]:
import os

# Initialize the RAG agent
agent = RAGAgent()

# Path to your PDF documents
pdf_directory = "/home/teng/development/projects/public/efh-hackathon/notebooks/data/Agroforestry"
# projects/public/efh-hackathon/notebooks/data
# projects/public/efh-hackathon/notebooks/data/Agroforestry
pdf_files = [
    os.path.join(pdf_directory, f) 
    for f in os.listdir(pdf_directory)
    if f.endswith('.pdf')
]

# Process the documents
print("Processing documents...")
agent.process_documents(pdf_files)

# Save the vector store for future use
agent.save_vector_store("vector_store")

# Test the RAG system
questions = [
    "What is the difference between the impact of trees in croplands and PES on food security?"
]

for question in questions:
    print(f"\nQuestion: {question}")
    response = agent.query(question)
    print("\nAnswer:", response["answer"])
    print("\nSources:")
    for doc in response["source_documents"]:
        print(f"  Source: {doc['source']}")
        print(f"  Relevant content: {doc['content'][:200]}...")

Processing documents...

Question: What is the difference between the impact of trees in croplands and PES on food security?


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

otherwise be provided in the absence of payment20. 
Protected Areas (PA): We applied the IUCN definition of Protected Areas which defines these as geographical 
areas dedicated to the protection and maintenance of biological diversity, and natural and associated cultural 
resources, and managed through legal or other effective means18. There were some overlaps in the Protected 
Areas and PES interventions in some of the included studies10,22 where PES programs were co-located within 
Protected Areas.   
Trees in croplands: This broadly refers to trees on cropland and includes the management of appr

In [36]:
import os

# Initialize the RAG agent
agent = RAGAgent()

# Path to your PDF documents
pdf_directory = "/home/teng/development/projects/public/efh-hackathon/notebooks/data/Agroforestry"
# projects/public/efh-hackathon/notebooks/data
# projects/public/efh-hackathon/notebooks/data/Agroforestry
pdf_files = [
    os.path.join(pdf_directory, f) 
    for f in os.listdir(pdf_directory)
    if f.endswith('.pdf')
]

# Process the documents
print("Processing documents...")
agent.process_documents(pdf_files)

# Save the vector store for future use
agent.save_vector_store("vector_store")

# Test the RAG system
questions = [
    "What are the most important impacts of tree-based interventions on health and wellbeing?"
]

for question in questions:
    print(f"\nQuestion: {question}")
    response = agent.query(question)
    print("\nAnswer:", response["answer"])
    print("\nSources:")
    for doc in response["source_documents"]:
        print(f"  Source: {doc['source']}")
        print(f"  Relevant content: {doc['content'][:200]}...")

Processing documents...

Question: What are the most important impacts of tree-based interventions on health and wellbeing?


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Correspondence to:
Dr Peninah Murage, Department 
of Public Health, Environments 
and Society, London School of 
Hygiene & Tropical Medicine, 
London WC1E 7HT, UK
peninah.murage@lshtm.ac.uk
Impact of tree-based interventions in addressing health and 
wellbeing outcomes in rural low-income and middle-income 
settings: a systematic review and meta-analysis
Peninah Murage, Blanca Anton, Faraja Chiwanga, Roberto Picetti, Tabby Njunge, Syreen Hassan, Sarah Whitmee, Jane Falconer, 
Hugh Sharma Waddington*, Rosemary Green*
The impact of nature-based solutions on human health is increasingly recognised; how

In [37]:
import os

# Initialize the RAG agent
agent = RAGAgent()

# Path to your PDF documents
pdf_directory = "/home/teng/development/projects/public/efh-hackathon/notebooks/data/Agroforestry"
# projects/public/efh-hackathon/notebooks/data
# projects/public/efh-hackathon/notebooks/data/Agroforestry
pdf_files = [
    os.path.join(pdf_directory, f) 
    for f in os.listdir(pdf_directory)
    if f.endswith('.pdf')
]

# Process the documents
print("Processing documents...")
agent.process_documents(pdf_files)

# Save the vector store for future use
agent.save_vector_store("vector_store")

# Test the RAG system
questions = [
    "How do different tree intervention practices impact food security?"
]

for question in questions:
    print(f"\nQuestion: {question}")
    response = agent.query(question)
    print("\nAnswer:", response["answer"])
    print("\nSources:")
    for doc in response["source_documents"]:
        print(f"  Source: {doc['source']}")
        print(f"  Relevant content: {doc['content'][:200]}...")

Processing documents...

Question: How do different tree intervention practices impact food security?


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

adjustment for dependence) are shown in the appendix 
(p 16).
For outcomes related to food and nutrition, the impacts 
of interventions on agricultural yields and food security 
were examined in ten studies (more than 20 different 
analyses; figure 3). The pooled effect across all 
interventions showed a significant, moderate-sized 
positive impact on agricultural yield (SMD 0·41 [95% CI 
0·11 to 0·70], p=0·0076; I²=94·9% and τ²=0·17; figure 3). 
The subgroup analysis found that interventions related 
to trees in croplands had the greatest significant effects 
(0·35 [0·04 to 0·67]; p=0·025; I²=95·2% and τ²=0·26). 
Smalle

In [42]:
import os
import warnings
from typing import List, Dict, Tuple
from pathlib import Path
import numpy as np
import fitz  # PyMuPDF
from dotenv import load_dotenv
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_together import Together as TogetherLLM
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter

# Disable tokenizers parallelism warning
os.environ["TOKENIZERS_PARALLELISM"] = "false"

class RAGAgent:
    def __init__(self, model_name: str = "mistralai/Mixtral-8x7B-Instruct-v0.1"):
        load_dotenv()
        
        # Initialize embeddings
        self.embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-mpnet-base-v2"
        )
        
        # Initialize text splitter
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len,
        )
        
        # Initialize LLM
        self.llm = TogetherLLM(
            api_key=TOGETHER_API_KEY,
            model=model_name,
            temperature=0.7,
            max_tokens=2048,
        )
        
        self.vector_store = None
        self.conversation_chain = None
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            output_key="answer",
            input_key="question",
            return_messages=True
        )

    def get_relevant_documents(self, question: str) -> Tuple[List[Document], List[float]]:
        """Get relevant documents with similarity scores."""
        if self.vector_store is None:
            return [], []
        
        # Get documents and scores from the vector store
        docs_and_scores = self.vector_store.similarity_search_with_score(
            question,
            k=3
        )
        
        # Separate documents and scores
        docs = [doc for doc, _ in docs_and_scores]
        scores = [score for _, score in docs_and_scores]
        
        return docs, scores

    def evaluate_response_quality(self, question: str, answer: str, context: str) -> float:
        """Evaluate the quality of the response based on relevance and completeness."""
        # Calculate cosine similarity between question and answer embeddings
        question_embedding = self.embeddings.embed_query(question)
        answer_embedding = self.embeddings.embed_query(answer)
        
        # Convert to numpy arrays for calculation
        q_embed = np.array(question_embedding)
        a_embed = np.array(answer_embedding)
        
        # Calculate cosine similarity
        similarity = np.dot(q_embed, a_embed) / (np.linalg.norm(q_embed) * np.linalg.norm(a_embed))
        
        return float(similarity)

    def process_pdf(self, pdf_path: str) -> str:
        """Extract text from PDF file using PyMuPDF."""
        text = ""
        with fitz.open(pdf_path) as doc:
            for page in doc:
                text += page.get_text()
        return text

    def process_documents(self, file_paths: List[str]) -> None:
        """Process multiple documents and create vector store."""
        documents = []
        
        for file_path in file_paths:
            if file_path.endswith('.pdf'):
                text = self.process_pdf(file_path)
                doc = Document(
                    page_content=text,
                    metadata={"source": file_path}
                )
                documents.append(doc)
        
        # Split documents into chunks
        splits = self.text_splitter.split_documents(documents)
        
        # Create or update vector store
        if self.vector_store is None:
            self.vector_store = FAISS.from_documents(splits, self.embeddings)
        else:
            self.vector_store.add_documents(splits)
        
        # Initialize conversation chain with enhanced retriever
        base_retriever = self.vector_store.as_retriever(
            search_kwargs={"k": 3}
        )
        
        # Add embeddings filter for better context relevance
        embeddings_filter = EmbeddingsFilter(
            embeddings=self.embeddings,
            similarity_threshold=0.7
        )
        compression_retriever = ContextualCompressionRetriever(
            base_compressor=embeddings_filter,
            base_retriever=base_retriever
        )
        
        self.conversation_chain = ConversationalRetrievalChain.from_llm(
            llm=self.llm,
            retriever=compression_retriever,
            memory=self.memory,
            return_source_documents=True,
            verbose=True,
            return_generated_question=False,
            chain_type="stuff",
            rephrase_question=False
        )

    def query(self, question: str) -> Dict:
        """Query the RAG system with a question."""
        if self.conversation_chain is None:
            raise ValueError("Please process documents before querying.")
        
        try:
            # Get relevant documents with scores
            docs, relevance_scores = self.get_relevant_documents(question)
            
            # Get the response from the conversation chain
            response = self.conversation_chain({"question": question})
            answer = response["answer"]
            
            # Evaluate response quality
            response_quality = self.evaluate_response_quality(
                question,
                answer,
                " ".join([doc.page_content for doc in docs])
            )
            
            return {
                "answer": answer,
                "source_documents": [
                    {
                        "content": doc.page_content,
                        "source": doc.metadata["source"],
                        "relevance_score": float(score)  # Convert to float for JSON serialization
                    }
                    for doc, score in zip(response["source_documents"], relevance_scores)
                ],
                "response_quality_score": response_quality,
                "average_relevance_score": float(np.mean(relevance_scores)) if relevance_scores else 0.0
            }
        except Exception as e:
            print(f"Error during query: {str(e)}")
            return {
                "answer": "Sorry, I encountered an error while processing your question.",
                "source_documents": [],
                "response_quality_score": 0.0,
                "average_relevance_score": 0.0
            }

    def save_vector_store(self, path: str) -> None:
        """Save the vector store to disk."""
        if self.vector_store is not None:
            self.vector_store.save_local(path)

    def load_vector_store(self, path: str) -> None:
        """Load the vector store from disk."""
        if os.path.exists(path):
            self.vector_store = FAISS.load_local(path, self.embeddings)
            self.conversation_chain = ConversationalRetrievalChain.from_llm(
                llm=self.llm,
                retriever=self.vector_store.as_retriever(
                    search_kwargs={"k": 3}
                ),
                memory=self.memory,
                return_source_documents=True,
            ) 

In [44]:
import os

# Initialize the RAG agent
agent = RAGAgent()

# Path to your PDF documents
pdf_directory = "/home/teng/development/projects/public/efh-hackathon/notebooks/data/Agroforestry"
# projects/public/efh-hackathon/notebooks/data
# projects/public/efh-hackathon/notebooks/data/Agroforestry
pdf_files = [
    os.path.join(pdf_directory, f) 
    for f in os.listdir(pdf_directory)
    if f.endswith('.pdf')
]

# Process the documents
print("Processing documents...")
agent.process_documents(pdf_files)

# Save the vector store for future use
agent.save_vector_store("vector_store")

# Test the RAG system
questions = [
    "How do different tree intervention practices impact food security?"
]

for question in questions:
    print(f"\nQuestion: {question}")
    response = agent.query(question)
    print("\nAnswer:", response["answer"])
    print(f"Response Quality Score: {response['response_quality_score']:.2f}")
    print(f"Average Document Relevance: {response['average_relevance_score']:.2f}\n")
    print("\nSource Documents:")
    for doc in response["source_documents"]:
        print(f"  Relevance Score: {doc['relevance_score']:.2f}")
        print(f"  Source: {doc['source']}")
        print(f"  Relevant content: {doc['content'][:200]}...")

Processing documents...

Question: How do different tree intervention practices impact food security?


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

adjustment for dependence) are shown in the appendix 
(p 16).
For outcomes related to food and nutrition, the impacts 
of interventions on agricultural yields and food security 
were examined in ten studies (more than 20 different 
analyses; figure 3). The pooled effect across all 
interventions showed a significant, moderate-sized 
positive impact on agricultural yield (SMD 0·41 [95% CI 
0·11 to 0·70], p=0·0076; I²=94·9% and τ²=0·17; figure 3). 
The subgroup analysis found that interventions related 
to trees in croplands had the greatest significant effects 
(0·35 [0·04 to 0·67]; p=0·025; I²=95·2% and τ²=0·26). 
Smalle

In [47]:
import os
import warnings
from typing import List, Dict, Tuple
from pathlib import Path
import numpy as np
import fitz  # PyMuPDF
from dotenv import load_dotenv
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_together import TogetherEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_together import Together as TogetherLLM
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter

# Disable tokenizers parallelism warning
os.environ["TOKENIZERS_PARALLELISM"] = "false"

class RAGAgent:
    def __init__(self, model_name: str = "mistralai/Mixtral-8x7B-Instruct-v0.1"):
        load_dotenv()
        
        # Initialize embeddings
        self.embeddings = TogetherEmbeddings(
            api_key=TOGETHER_API_KEY,
            model="togethercomputer/m2-bert-80M-8k-retrieval",
        )
        
        # Initialize text splitter
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len,
        )
        
        # Initialize LLM
        self.llm = TogetherLLM(
            api_key=TOGETHER_API_KEY,
            model=model_name,
            temperature=0.7,
            max_tokens=2048,
        )
        
        self.vector_store = None
        self.conversation_chain = None
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            output_key="answer",
            input_key="question",
            return_messages=True
        )

    def get_relevant_documents(self, question: str) -> Tuple[List[Document], List[float]]:
        """Get relevant documents with similarity scores."""
        if self.vector_store is None:
            return [], []
        
        # Get documents and scores from the vector store
        docs_and_scores = self.vector_store.similarity_search_with_score(
            question,
            k=3
        )
        
        # Separate documents and scores
        docs = [doc for doc, _ in docs_and_scores]
        scores = [score for _, score in docs_and_scores]
        
        return docs, scores

    def evaluate_response_quality(self, question: str, answer: str, context: str) -> float:
        """Evaluate the quality of the response based on relevance and completeness."""
        # Calculate cosine similarity between question and answer embeddings
        question_embedding = self.embeddings.embed_query(question)
        answer_embedding = self.embeddings.embed_query(answer)
        
        # Convert to numpy arrays for calculation
        q_embed = np.array(question_embedding)
        a_embed = np.array(answer_embedding)
        
        # Calculate cosine similarity
        similarity = np.dot(q_embed, a_embed) / (np.linalg.norm(q_embed) * np.linalg.norm(a_embed))
        
        return float(similarity)

    def process_pdf(self, pdf_path: str) -> str:
        """Extract text from PDF file using PyMuPDF."""
        text = ""
        with fitz.open(pdf_path) as doc:
            for page in doc:
                text += page.get_text()
        return text

    def process_documents(self, file_paths: List[str]) -> None:
        """Process multiple documents and create vector store."""
        documents = []
        
        for file_path in file_paths:
            if file_path.endswith('.pdf'):
                text = self.process_pdf(file_path)
                doc = Document(
                    page_content=text,
                    metadata={"source": file_path}
                )
                documents.append(doc)
        
        # Split documents into chunks
        splits = self.text_splitter.split_documents(documents)
        
        # Create or update vector store
        if self.vector_store is None:
            self.vector_store = FAISS.from_documents(splits, self.embeddings)
        else:
            self.vector_store.add_documents(splits)
        
        # Initialize conversation chain with enhanced retriever
        base_retriever = self.vector_store.as_retriever(
            search_kwargs={"k": 3}
        )
        
        # Add embeddings filter for better context relevance
        embeddings_filter = EmbeddingsFilter(
            embeddings=self.embeddings,
            similarity_threshold=0.7
        )
        compression_retriever = ContextualCompressionRetriever(
            base_compressor=embeddings_filter,
            base_retriever=base_retriever
        )
        
        self.conversation_chain = ConversationalRetrievalChain.from_llm(
            llm=self.llm,
            retriever=compression_retriever,
            memory=self.memory,
            return_source_documents=True,
            verbose=True,
            return_generated_question=False,
            chain_type="stuff",
            rephrase_question=False
        )

    def query(self, question: str) -> Dict:
        """Query the RAG system with a question."""
        if self.conversation_chain is None:
            raise ValueError("Please process documents before querying.")
        
        try:
            # Get relevant documents with scores
            docs, relevance_scores = self.get_relevant_documents(question)
            
            # Get the response from the conversation chain
            response = self.conversation_chain({"question": question})
            answer = response["answer"]
            
            # Evaluate response quality
            response_quality = self.evaluate_response_quality(
                question,
                answer,
                " ".join([doc.page_content for doc in docs])
            )
            
            return {
                "answer": answer,
                "source_documents": [
                    {
                        "content": doc.page_content,
                        "source": doc.metadata["source"],
                        "relevance_score": float(score)  # Convert to float for JSON serialization
                    }
                    for doc, score in zip(response["source_documents"], relevance_scores)
                ],
                "response_quality_score": response_quality,
                "average_relevance_score": float(np.mean(relevance_scores)) if relevance_scores else 0.0
            }
        except Exception as e:
            print(f"Error during query: {str(e)}")
            return {
                "answer": "Sorry, I encountered an error while processing your question.",
                "source_documents": [],
                "response_quality_score": 0.0,
                "average_relevance_score": 0.0
            }

    def save_vector_store(self, path: str) -> None:
        """Save the vector store to disk."""
        if self.vector_store is not None:
            self.vector_store.save_local(path)

    def load_vector_store(self, path: str) -> None:
        """Load the vector store from disk."""
        if os.path.exists(path):
            self.vector_store = FAISS.load_local(path, self.embeddings)
            self.conversation_chain = ConversationalRetrievalChain.from_llm(
                llm=self.llm,
                retriever=self.vector_store.as_retriever(
                    search_kwargs={"k": 3}
                ),
                memory=self.memory,
                return_source_documents=True,
            ) 

In [48]:
import os

# Initialize the RAG agent
agent = RAGAgent()

# Path to your PDF documents
pdf_directory = "/home/teng/development/projects/public/efh-hackathon/notebooks/data/Agroforestry"
# projects/public/efh-hackathon/notebooks/data
# projects/public/efh-hackathon/notebooks/data/Agroforestry
pdf_files = [
    os.path.join(pdf_directory, f) 
    for f in os.listdir(pdf_directory)
    if f.endswith('.pdf')
]

# Process the documents
print("Processing documents...")
agent.process_documents(pdf_files)

# Save the vector store for future use
agent.save_vector_store("vector_store")

# Test the RAG system
questions = [
    "How do different tree intervention practices impact food security?"
]

for question in questions:
    print(f"\nQuestion: {question}")
    response = agent.query(question)
    print("\nAnswer:", response["answer"])
    print(f"Response Quality Score: {response['response_quality_score']:.2f}")
    print(f"Average Document Relevance: {response['average_relevance_score']:.2f}\n")
    print("\nSource Documents:")
    for doc in response["source_documents"]:
        print(f"  Relevance Score: {doc['relevance_score']:.2f}")
        print(f"  Source: {doc['source']}")
        print(f"  Relevant content: {doc['content'][:200]}...")

Processing documents...

Question: How do different tree intervention practices impact food security?


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

from four analyses from two studies (figure 5). One study 
found a significant reduction in hospital admissions 
attributed to a PES intervention; this improvement in 
health probably contributed to increases in agricultural 
productivity.79
Egger’s test did not indicate funnel plot asymmetry and 
therefore did not suggest missing or unreported evidence 
(appendix pp 17–18), although some outcomes were 
reported in too few studies to enable funnel plot analyses 
using Egger’s regression.
Discussion 
Our meta-analysis, based on 54 studies, found that tree-
based conservation solutions can have positive and 
significant im

In [57]:
import os
import warnings
from typing import List, Dict, Tuple
from pathlib import Path
import numpy as np
import fitz  # PyMuPDF
from dotenv import load_dotenv
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_together import TogetherEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_together import Together as TogetherLLM
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter

# Disable tokenizers parallelism warning
os.environ["TOKENIZERS_PARALLELISM"] = "false"

class RAGAgent:
    def __init__(self, model_name: str = "Qwen/Qwen2-72B-Instruct"):
        load_dotenv()
        
        # Initialize embeddings
        self.embeddings = TogetherEmbeddings(
            api_key=TOGETHER_API_KEY,
            model="BAAI/bge-large-en-v1.5",
        )
        
        # Initialize text splitter
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=200,
            chunk_overlap=20,
            length_function=len,
        )
        
        # Initialize LLM
        self.llm = TogetherLLM(
            api_key=TOGETHER_API_KEY,
            model=model_name,
            temperature=0.7,
            max_tokens=256,
        )
        
        self.vector_store = None
        self.conversation_chain = None
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            output_key="answer",
            input_key="question",
            return_messages=True
        )

    def get_relevant_documents(self, question: str) -> Tuple[List[Document], List[float]]:
        """Get relevant documents with similarity scores."""
        if self.vector_store is None:
            return [], []
        
        # Get documents and scores from the vector store
        docs_and_scores = self.vector_store.similarity_search_with_score(
            question,
            k=3
        )
        
        # Separate documents and scores
        docs = [doc for doc, _ in docs_and_scores]
        scores = [score for _, score in docs_and_scores]
        
        return docs, scores

    def evaluate_response_quality(self, question: str, answer: str, context: str) -> float:
        """Evaluate the quality of the response based on relevance and completeness."""
        # Calculate cosine similarity between question and answer embeddings
        question_embedding = self.embeddings.embed_query(question)
        answer_embedding = self.embeddings.embed_query(answer)
        
        # Convert to numpy arrays for calculation
        q_embed = np.array(question_embedding)
        a_embed = np.array(answer_embedding)
        
        # Calculate cosine similarity
        similarity = np.dot(q_embed, a_embed) / (np.linalg.norm(q_embed) * np.linalg.norm(a_embed))
        
        return float(similarity)
    
    def batch_embed_documents(self, texts: List[str], batch_size: int = 8) -> List[List[float]]:
        """Embed documents in batches to handle token limits."""
        all_embeddings = []
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i + batch_size]
            try:
                batch_embeddings = self.embeddings.embed_documents(batch)
                all_embeddings.extend(batch_embeddings)
            except Exception as e:
                print(f"Error embedding batch {i//batch_size}: {str(e)}")
                # Use zero embeddings as fallback for failed batch
                for _ in range(len(batch)):
                    all_embeddings.append([0.0] * 1024)  # Assuming 1024-dimensional embeddings
        return all_embeddings

    def process_pdf(self, pdf_path: str) -> str:
        """Extract text from PDF file using PyMuPDF."""
        text = ""
        with fitz.open(pdf_path) as doc:
            for page in doc:
                text += page.get_text()
        return text

    def process_documents(self, file_paths: List[str]) -> None:
        """Process multiple documents and create vector store."""
        documents = []
        
        for file_path in file_paths:
            if file_path.endswith('.pdf'):
                text = self.process_pdf(file_path)
                doc = Document(
                    page_content=text,
                    metadata={"source": file_path}
                )
                documents.append(doc)
        
        # Split documents into chunks
        splits = self.text_splitter.split_documents(documents)
        
        # Create or update vector store with batched processing
        texts = [doc.page_content for doc in splits]
        metadatas = [doc.metadata for doc in splits]
        
        # Create embeddings in batches
        embeddings = self.batch_embed_documents(texts)
        
        # Create or update vector store
        if self.vector_store is None:
            self.vector_store = FAISS.from_embeddings(
                text_embeddings=list(zip(texts, embeddings)),
                embedding=self.embeddings,
                metadatas=metadatas
            )
        else:
            self.vector_store.add_embeddings(
                text_embeddings=list(zip(texts, embeddings)),
                metadatas=metadatas
            )
        
        # Initialize conversation chain with enhanced retriever
        base_retriever = self.vector_store.as_retriever(
            search_kwargs={"k": 3}
        )
        
        # Add embeddings filter for better context relevance
        embeddings_filter = EmbeddingsFilter(
            embeddings=self.embeddings,
            similarity_threshold=0.7
        )
        compression_retriever = ContextualCompressionRetriever(
            base_compressor=embeddings_filter,
            base_retriever=base_retriever
        )
        
        self.conversation_chain = ConversationalRetrievalChain.from_llm(
            llm=self.llm,
            retriever=compression_retriever,
            memory=self.memory,
            return_source_documents=True,
            verbose=True,
            return_generated_question=False,
            chain_type="stuff",
            rephrase_question=False
        )

    def truncate_context(self, context: str, max_tokens: int = 400) -> str:
        """Truncate context to fit within token limit."""
        words = context.split()
        if len(words) > max_tokens:
            return ' '.join(words[:max_tokens])
        return context

    def query(self, question: str) -> Dict:
        """Query the RAG system with a question."""
        if self.conversation_chain is None:
            raise ValueError("Please process documents before querying.")
        
        try:
            # Get relevant documents with scores
            docs, relevance_scores = self.get_relevant_documents(question)
            
            # Truncate context to fit within token limits
            truncated_docs = [
                Document(
                    page_content=self.truncate_context(doc.page_content),
                    metadata=doc.metadata
                )
                for doc in docs
            ]
            
            # Update retriever with truncated documents
            self.conversation_chain.retriever.document_store = FAISS.from_documents(
                truncated_docs,
                self.embeddings
            )
            
            # Get the response from the conversation chain
            response = self.conversation_chain({"question": question})
            answer = response["answer"]
            
            # Evaluate response quality
            response_quality = self.evaluate_response_quality(
                question,
                answer,
                " ".join([doc.page_content for doc in truncated_docs])
            )
            
            return {
                "answer": answer,
                "source_documents": [
                    {
                        "content": doc.page_content,
                        "source": doc.metadata["source"],
                        "relevance_score": float(score)
                    }
                    for doc, score in zip(response["source_documents"], relevance_scores)
                ],
                "response_quality_score": response_quality,
                "average_relevance_score": float(np.mean(relevance_scores)) if relevance_scores else 0.0
            }
        except Exception as e:
            print(f"Error during query: {str(e)}")
            return {
                "answer": "Sorry, I encountered an error while processing your question.",
                "source_documents": [],
                "response_quality_score": 0.0,
                "average_relevance_score": 0.0
            }

    def save_vector_store(self, path: str) -> None:
        """Save the vector store to disk."""
        if self.vector_store is not None:
            self.vector_store.save_local(path)

    def load_vector_store(self, path: str) -> None:
        """Load the vector store from disk."""
        if os.path.exists(path):
            self.vector_store = FAISS.load_local(path, self.embeddings)
            self.conversation_chain = ConversationalRetrievalChain.from_llm(
                llm=self.llm,
                retriever=self.vector_store.as_retriever(
                    search_kwargs={"k": 3}
                ),
                memory=self.memory,
                return_source_documents=True,
            ) 

In [58]:
import os

# Initialize the RAG agent
agent = RAGAgent()

# Path to your PDF documents
pdf_directory = "/home/teng/development/projects/public/efh-hackathon/notebooks/data/Agroforestry"

pdf_files = [
    os.path.join(pdf_directory, f) 
    for f in os.listdir(pdf_directory)
    if f.endswith('.pdf')
]

# Process the documents
print("Processing documents...")
agent.process_documents(pdf_files)

# Save the vector store for future use
agent.save_vector_store("vector_store")

# Test the RAG system
questions = [
    "How do different tree intervention practices impact food security?"
]

for question in questions:
    print(f"\nQuestion: {question}")
    response = agent.query(question)
    print("\nAnswer:", response["answer"])
    print(f"Response Quality Score: {response['response_quality_score']:.2f}")
    print(f"Average Document Relevance: {response['average_relevance_score']:.2f}\n")
    print("\nSource Documents:")
    for doc in response["source_documents"]:
        print(f"  Relevance Score: {doc['relevance_score']:.2f}")
        print(f"  Source: {doc['source']}")
        print(f"  Relevant content: {doc['content'][:200]}...")

Processing documents...

Question: How do different tree intervention practices impact food security?
Error during query: "ContextualCompressionRetriever" object has no field "document_store"

Answer: Sorry, I encountered an error while processing your question.
Response Quality Score: 0.00
Average Document Relevance: 0.00


Source Documents:


In [59]:
import os
import warnings
from typing import List, Dict, Tuple
from pathlib import Path
import numpy as np
import fitz  # PyMuPDF
from dotenv import load_dotenv
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_together import TogetherEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_together import Together as TogetherLLM
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter
from concurrent.futures import ThreadPoolExecutor
from functools import lru_cache
import time

class RAGAgent:
    def __init__(self, model_name: str = "Qwen/Qwen2-72B-Instruct"):
        load_dotenv()
        
        # Initialize embeddings with caching
        self.embeddings = TogetherEmbeddings(
            api_key=TOGETHER_API_KEY,
            model="togethercomputer/m2-bert-80M-8k-retrieval",  # Smaller, faster model
            cache_folder="./embeddings_cache"
        )
        
        # Initialize text splitter with optimized parameters
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=200,
            chunk_overlap=20,
            length_function=len,
        )
        
        # Initialize LLM with optimized parameters
        self.llm = TogetherLLM(
            api_key=TOGETHER_API_KEY,
            model=model_name,
            temperature=0.7,
            max_tokens=256,
            request_timeout=30,
        )
        
        self.vector_store = None
        self.conversation_chain = None
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            output_key="answer",
            input_key="question",
            return_messages=True
        )
        
        # Initialize cache for document processing
        self.doc_cache = {}

    @lru_cache(maxsize=1000)
    def get_embedding(self, text: str) -> List[float]:
        """Cached embedding generation."""
        return self.embeddings.embed_query(text)

    def process_pdf_parallel(self, file_paths: List[str]) -> List[Document]:
        """Process PDFs in parallel."""
        def process_single_pdf(file_path: str) -> Document:
            # Check cache first
            if file_path in self.doc_cache:
                return self.doc_cache[file_path]
            
            text = ""
            with fitz.open(file_path) as doc:
                text = " ".join(page.get_text() for page in doc)
            
            document = Document(
                page_content=text,
                metadata={"source": file_path}
            )
            self.doc_cache[file_path] = document
            return document

        with ThreadPoolExecutor(max_workers=4) as executor:
            documents = list(executor.map(process_single_pdf, file_paths))
        return documents

    def batch_embed_documents(self, texts: List[str], batch_size: int = 8) -> List[List[float]]:
        """Optimized batch embedding with caching."""
        all_embeddings = []
        
        # Process in batches
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i + batch_size]
            batch_embeddings = []
            
            for text in batch:
                # Try to get from cache first
                embedding = self.get_embedding(text)
                batch_embeddings.append(embedding)
            
            all_embeddings.extend(batch_embeddings)
        
        return all_embeddings

    def process_documents(self, file_paths: List[str]) -> None:
        """Optimized document processing."""
        start_time = time.time()
        print("Starting document processing...")
        
        # Process PDFs in parallel
        documents = self.process_pdf_parallel(file_paths)
        
        # Split documents into chunks
        splits = self.text_splitter.split_documents(documents)
        
        # Create or update vector store with batched processing
        texts = [doc.page_content for doc in splits]
        metadatas = [doc.metadata for doc in splits]
        
        print(f"Processing {len(texts)} text chunks...")
        
        # Create embeddings in optimized batches
        embeddings = self.batch_embed_documents(texts, batch_size=16)
        
        # Create or update vector store
        if self.vector_store is None:
            self.vector_store = FAISS.from_embeddings(
                text_embeddings=list(zip(texts, embeddings)),
                embedding=self.embeddings,
                metadatas=metadatas
            )
        else:
            self.vector_store.add_embeddings(
                text_embeddings=list(zip(texts, embeddings)),
                metadatas=metadatas
            )
        
        # Initialize conversation chain with optimized retriever
        base_retriever = self.vector_store.as_retriever(
            search_kwargs={"k": 2}  # Reduced from 3 for faster retrieval
        )
        
        self.conversation_chain = ConversationalRetrievalChain.from_llm(
            llm=self.llm,
            retriever=base_retriever,
            memory=self.memory,
            return_source_documents=True,
            verbose=False,  # Reduced logging for better performance
            return_generated_question=False,
            chain_type="stuff",
        )
        
        end_time = time.time()
        print(f"Document processing completed in {end_time - start_time:.2f} seconds")

    def query(self, question: str) -> Dict:
        """Optimized query processing."""
        if self.conversation_chain is None:
            raise ValueError("Please process documents before querying.")
        
        try:
            start_time = time.time()
            
            # Get relevant documents with scores
            docs, relevance_scores = self.get_relevant_documents(question)
            
            # Get the response from the conversation chain
            response = self.conversation_chain({"question": question})
            answer = response["answer"]
            
            # Calculate response quality
            response_quality = self.evaluate_response_quality(
                question,
                answer,
                " ".join([doc.page_content[:200] for doc in docs])  # Limit context size
            )
            
            end_time = time.time()
            print(f"Query processed in {end_time - start_time:.2f} seconds")
            
            return {
                "answer": answer,
                "source_documents": [
                    {
                        "content": doc.page_content[:200],  # Limit content size
                        "source": doc.metadata["source"],
                        "relevance_score": float(score)
                    }
                    for doc, score in zip(response["source_documents"], relevance_scores)
                ],
                "response_quality_score": response_quality,
                "average_relevance_score": float(np.mean(relevance_scores)) if relevance_scores else 0.0
            }
        except Exception as e:
            print(f"Error during query: {str(e)}")
            return {
                "answer": "Sorry, I encountered an error while processing your question.",
                "source_documents": [],
                "response_quality_score": 0.0,
                "average_relevance_score": 0.0
            }

In [60]:
import os

# Initialize the RAG agent
agent = RAGAgent()

# Path to your PDF documents
pdf_directory = "/home/teng/development/projects/public/efh-hackathon/notebooks/data/Agroforestry"
pdf_files = [
    os.path.join(pdf_directory, f) 
    for f in os.listdir(pdf_directory)
    if f.endswith('.pdf')
]

# Process the documents
print("Processing documents...")
agent.process_documents(pdf_files)

# Test the RAG system
questions = [
    "How do different tree intervention practices impact food security?"
]

for question in questions:
    print(f"\nQuestion: {question}")
    response = agent.query(question)
    print("\nAnswer:", response["answer"])
    print(f"Response Quality Score: {response['response_quality_score']:.2f}")
    print(f"Average Document Relevance: {response['average_relevance_score']:.2f}\n")
    print("\nSource Documents:")
    for doc in response["source_documents"]:
        print(f"  Relevance Score: {doc['relevance_score']:.2f}")
        print(f"  Source: {doc['source']}")
        print(f"  Relevant content: {doc['content']}")

/home/teng/miniconda/lib/python3.10/site-packages/langchain_together/embeddings.py:191: UserWarning: WARNING! cache_folder is not default parameter.
                    cache_folder was transferred to model_kwargs.
                    Please confirm that cache_folder is what you intended.
  warnings.warn(


ValidationError: 1 validation error for Together
request_timeout
  Extra inputs are not permitted [type=extra_forbidden, input_value=30, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/extra_forbidden

In [61]:
import os
import warnings
from typing import List, Dict, Tuple
from pathlib import Path
import numpy as np
import fitz  # PyMuPDF
from dotenv import load_dotenv
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_together import Together as TogetherLLM
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter

# Disable tokenizers parallelism warning
os.environ["TOKENIZERS_PARALLELISM"] = "false"

class RAGAgent:
    def __init__(self, model_name: str = "mistralai/Mixtral-8x7B-Instruct-v0.1"):
        load_dotenv()
        
        # Initialize embeddings
        self.embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-mpnet-base-v2"
        )
        
        # Initialize text splitter
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len,
        )
        
        # Initialize LLM
        self.llm = TogetherLLM(
            api_key=TOGETHER_API_KEY,
            model=model_name,
            temperature=0.7,
            max_tokens=2048,
        )
        
        self.vector_store = None
        self.conversation_chain = None
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            output_key="answer",
            input_key="question",
            return_messages=True
        )

    def get_relevant_documents(self, question: str) -> Tuple[List[Document], List[float]]:
        """Get relevant documents with similarity scores."""
        if self.vector_store is None:
            return [], []
        
        # Get documents and scores from the vector store
        docs_and_scores = self.vector_store.similarity_search_with_score(
            question,
            k=3
        )
        
        # Separate documents and scores
        docs = [doc for doc, _ in docs_and_scores]
        scores = [score for _, score in docs_and_scores]
        
        return docs, scores

    def evaluate_response_quality(self, question: str, answer: str, context: str) -> float:
        """Evaluate the quality of the response based on relevance and completeness."""
        # Calculate cosine similarity between question and answer embeddings
        question_embedding = self.embeddings.embed_query(question)
        answer_embedding = self.embeddings.embed_query(answer)
        
        # Convert to numpy arrays for calculation
        q_embed = np.array(question_embedding)
        a_embed = np.array(answer_embedding)
        
        # Calculate cosine similarity
        similarity = np.dot(q_embed, a_embed) / (np.linalg.norm(q_embed) * np.linalg.norm(a_embed))
        
        return float(similarity)

    def process_pdf(self, pdf_path: str) -> str:
        """Extract text from PDF file using PyMuPDF."""
        text = ""
        with fitz.open(pdf_path) as doc:
            for page in doc:
                text += page.get_text()
        return text

    def process_documents(self, file_paths: List[str]) -> None:
        """Process multiple documents and create vector store."""
        documents = []
        
        for file_path in file_paths:
            if file_path.endswith('.pdf'):
                text = self.process_pdf(file_path)
                doc = Document(
                    page_content=text,
                    metadata={"source": file_path}
                )
                documents.append(doc)
        
        # Split documents into chunks
        splits = self.text_splitter.split_documents(documents)
        
        # Create or update vector store
        if self.vector_store is None:
            self.vector_store = FAISS.from_documents(splits, self.embeddings)
        else:
            self.vector_store.add_documents(splits)
        
        # Initialize conversation chain with enhanced retriever
        base_retriever = self.vector_store.as_retriever(
            search_kwargs={"k": 3}
        )
        
        # Add embeddings filter for better context relevance
        embeddings_filter = EmbeddingsFilter(
            embeddings=self.embeddings,
            similarity_threshold=0.7
        )
        compression_retriever = ContextualCompressionRetriever(
            base_compressor=embeddings_filter,
            base_retriever=base_retriever
        )
        
        self.conversation_chain = ConversationalRetrievalChain.from_llm(
            llm=self.llm,
            retriever=compression_retriever,
            memory=self.memory,
            return_source_documents=True,
            verbose=True,
            return_generated_question=False,
            chain_type="stuff",
            rephrase_question=False
        )

    def query(self, question: str) -> Dict:
        """Query the RAG system with a question."""
        if self.conversation_chain is None:
            raise ValueError("Please process documents before querying.")
        
        try:
            # Get relevant documents with scores
            docs, relevance_scores = self.get_relevant_documents(question)
            
            # Get the response from the conversation chain
            response = self.conversation_chain({"question": question})
            answer = response["answer"]
            
            # Evaluate response quality
            response_quality = self.evaluate_response_quality(
                question,
                answer,
                " ".join([doc.page_content for doc in docs])
            )
            
            return {
                "answer": answer,
                "source_documents": [
                    {
                        "content": doc.page_content,
                        "source": doc.metadata["source"],
                        "relevance_score": float(score)  # Convert to float for JSON serialization
                    }
                    for doc, score in zip(response["source_documents"], relevance_scores)
                ],
                "response_quality_score": response_quality,
                "average_relevance_score": float(np.mean(relevance_scores)) if relevance_scores else 0.0
            }
        except Exception as e:
            print(f"Error during query: {str(e)}")
            return {
                "answer": "Sorry, I encountered an error while processing your question.",
                "source_documents": [],
                "response_quality_score": 0.0,
                "average_relevance_score": 0.0
            }

    def save_vector_store(self, path: str) -> None:
        """Save the vector store to disk."""
        if self.vector_store is not None:
            self.vector_store.save_local(path)

    def load_vector_store(self, path: str) -> None:
        """Load the vector store from disk."""
        if os.path.exists(path):
            self.vector_store = FAISS.load_local(path, self.embeddings)
            self.conversation_chain = ConversationalRetrievalChain.from_llm(
                llm=self.llm,
                retriever=self.vector_store.as_retriever(
                    search_kwargs={"k": 3}
                ),
                memory=self.memory,
                return_source_documents=True,
            ) 

In [62]:
import os

# Initialize the RAG agent
agent = RAGAgent()

# Path to your PDF documents
pdf_directory = "/home/teng/development/projects/public/efh-hackathon/notebooks/data/Agroforestry"

pdf_files = [
    os.path.join(pdf_directory, f) 
    for f in os.listdir(pdf_directory)
    if f.endswith('.pdf')
]

# Process the documents
print("Processing documents...")
agent.process_documents(pdf_files)

# Save the vector store for future use
agent.save_vector_store("vector_store")

# Test the RAG system
questions = [
    "Can you identify and rank potential environmental levers for improved health outcomes?"
]

for question in questions:
    print(f"\nQuestion: {question}")
    response = agent.query(question)
    print("\nAnswer:", response["answer"])
    print(f"Response Quality Score: {response['response_quality_score']:.2f}")
    print(f"Average Document Relevance: {response['average_relevance_score']:.2f}\n")
    print("\nSource Documents:")
    for doc in response["source_documents"]:
        print(f"  Relevance Score: {doc['relevance_score']:.2f}")
        print(f"  Source: {doc['source']}")
        print(f"  Relevant content: {doc['content'][:200]}...")

Processing documents...

Question: Can you identify and rank potential environmental levers for improved health outcomes?


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.



Question: Can you identify and rank potential environmental levers for improved health outcomes?
Helpful Answer:

> Finished chain.

> Finished chain.

Answer:  Environmental levers for improved health outcomes include water quality, air quality, waste management, and green spaces. The most important environmental lever would likely be air quality, as poor air quality can lead to respiratory issues and other health problems. Green spaces would be the second most important environmental lever, as research has shown that access to green spaces can improve mental health and promote physical activity. W

In [63]:
import os
import warnings
from typing import List, Dict, Tuple
from pathlib import Path
import numpy as np
import fitz  # PyMuPDF
from dotenv import load_dotenv
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_together import Together as TogetherLLM
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter

# Disable tokenizers parallelism warning
os.environ["TOKENIZERS_PARALLELISM"] = "false"

class RAGAgent:
    def __init__(self, model_name: str = "mistralai/Mixtral-8x7B-Instruct-v0.1"):
        load_dotenv()
        
        # Initialize embeddings
        self.embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-mpnet-base-v2"
        )
        
        # Initialize text splitter
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len,
        )
        
        # Initialize LLM
        self.llm = TogetherLLM(
            api_key=TOGETHER_API_KEY,
            model=model_name,
            temperature=0.7,
            max_tokens=2048,
        )
        
        self.vector_store = None
        self.conversation_chain = None
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            output_key="answer",
            input_key="question",
            return_messages=True
        )

    def get_relevant_documents(self, question: str) -> Tuple[List[Document], List[float]]:
        """Get relevant documents with similarity scores."""
        if self.vector_store is None:
            return [], []
        
        # Get documents and scores from the vector store
        docs_and_scores = self.vector_store.similarity_search_with_score(
            question,
            k=3
        )
        
        # Separate documents and scores
        docs = [doc for doc, _ in docs_and_scores]
        scores = [score for _, score in docs_and_scores]
        
        return docs, scores

    def evaluate_response_quality(self, question: str, answer: str, context: str) -> float:
        """Evaluate the quality of the response based on relevance and completeness."""
        # Calculate cosine similarity between question and answer embeddings
        question_embedding = self.embeddings.embed_query(question)
        answer_embedding = self.embeddings.embed_query(answer)
        
        # Convert to numpy arrays for calculation
        q_embed = np.array(question_embedding)
        a_embed = np.array(answer_embedding)
        
        # Calculate cosine similarity
        similarity = np.dot(q_embed, a_embed) / (np.linalg.norm(q_embed) * np.linalg.norm(a_embed))
        
        return float(similarity)

    def process_pdf(self, pdf_path: str) -> str:
        """Extract text from PDF file using PyMuPDF."""
        text = ""
        with fitz.open(pdf_path) as doc:
            for page in doc:
                text += page.get_text()
        return text

    def process_documents(self, file_paths: List[str]) -> None:
        """Process multiple documents and create vector store."""
        documents = []
        
        for file_path in file_paths:
            if file_path.endswith('.pdf'):
                text = self.process_pdf(file_path)
                doc = Document(
                    page_content=text,
                    metadata={"source": file_path}
                )
                documents.append(doc)
        
        # Split documents into chunks
        splits = self.text_splitter.split_documents(documents)
        
        # Create or update vector store
        if self.vector_store is None:
            self.vector_store = FAISS.from_documents(splits, self.embeddings)
        else:
            self.vector_store.add_documents(splits)
        
        # Initialize conversation chain with enhanced retriever
        base_retriever = self.vector_store.as_retriever(
            search_kwargs={"k": 2}
        )
        
        # Add embeddings filter for better context relevance
        embeddings_filter = EmbeddingsFilter(
            embeddings=self.embeddings,
            similarity_threshold=0.7
        )
        compression_retriever = ContextualCompressionRetriever(
            base_compressor=embeddings_filter,
            base_retriever=base_retriever
        )
        
        self.conversation_chain = ConversationalRetrievalChain.from_llm(
            llm=self.llm,
            retriever=compression_retriever,
            memory=self.memory,
            return_source_documents=True,
            verbose=True,
            return_generated_question=False,
            chain_type="stuff",
            rephrase_question=False
        )

    def query(self, question: str) -> Dict:
        """Query the RAG system with a question."""
        if self.conversation_chain is None:
            raise ValueError("Please process documents before querying.")
        
        try:
            # Get relevant documents with scores
            docs, relevance_scores = self.get_relevant_documents(question)
            
            # Get the response from the conversation chain
            response = self.conversation_chain({"question": question})
            answer = response["answer"]
            
            # Evaluate response quality
            response_quality = self.evaluate_response_quality(
                question,
                answer,
                " ".join([doc.page_content for doc in docs])
            )
            
            return {
                "answer": answer,
                "source_documents": [
                    {
                        "content": doc.page_content,
                        "source": doc.metadata["source"],
                        "relevance_score": float(score)  # Convert to float for JSON serialization
                    }
                    for doc, score in zip(response["source_documents"], relevance_scores)
                ],
                "response_quality_score": response_quality,
                "average_relevance_score": float(np.mean(relevance_scores)) if relevance_scores else 0.0
            }
        except Exception as e:
            print(f"Error during query: {str(e)}")
            return {
                "answer": "Sorry, I encountered an error while processing your question.",
                "source_documents": [],
                "response_quality_score": 0.0,
                "average_relevance_score": 0.0
            }

    def save_vector_store(self, path: str) -> None:
        """Save the vector store to disk."""
        if self.vector_store is not None:
            self.vector_store.save_local(path)

    def load_vector_store(self, path: str) -> None:
        """Load the vector store from disk."""
        if os.path.exists(path):
            self.vector_store = FAISS.load_local(path, self.embeddings)
            self.conversation_chain = ConversationalRetrievalChain.from_llm(
                llm=self.llm,
                retriever=self.vector_store.as_retriever(
                    search_kwargs={"k": 2}
                ),
                memory=self.memory,
                return_source_documents=True,
            ) 

In [64]:
import os

# Initialize the RAG agent
agent = RAGAgent()

# Path to your PDF documents
pdf_directory = "/home/teng/development/projects/public/efh-hackathon/notebooks/data/Agroforestry"

pdf_files = [
    os.path.join(pdf_directory, f) 
    for f in os.listdir(pdf_directory)
    if f.endswith('.pdf')
]

# Process the documents
print("Processing documents...")
agent.process_documents(pdf_files)

# Save the vector store for future use
agent.save_vector_store("vector_store")

# Test the RAG system
questions = [
    "Can you identify and rank potential environmental levers for improved health outcomes?"
]

for question in questions:
    print(f"\nQuestion: {question}")
    response = agent.query(question)
    print("\nAnswer:", response["answer"])
    print(f"Response Quality Score: {response['response_quality_score']:.2f}")
    print(f"Average Document Relevance: {response['average_relevance_score']:.2f}\n")
    print("\nSource Documents:")
    for doc in response["source_documents"]:
        print(f"  Relevance Score: {doc['relevance_score']:.2f}")
        print(f"  Source: {doc['source']}")
        print(f"  Relevant content: {doc['content'][:200]}...")

Processing documents...

Question: Can you identify and rank potential environmental levers for improved health outcomes?


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.



Question: Can you identify and rank potential environmental levers for improved health outcomes?
Helpful Answer:

> Finished chain.

> Finished chain.

Answer:  When thinking about ways to improve health outcomes, it's important to consider both individual behaviors and broader environmental factors. Here are some potential environmental levers that could be targeted:

1. Access to healthy food options: Policies that increase access to fresh fruits and vegetables, whole grains, and lean proteins can help people make healthier food choices.

2. Safe and walkable neighborhoods: Creating environments t

In [65]:
import os
import warnings
from typing import List, Dict, Tuple
from pathlib import Path
import numpy as np
import fitz  # PyMuPDF
from dotenv import load_dotenv
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_together import Together as TogetherLLM
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter

# Disable tokenizers parallelism warning
os.environ["TOKENIZERS_PARALLELISM"] = "false"

class RAGAgent:
    def __init__(self, model_name: str = "mistralai/Mixtral-8x7B-Instruct-v0.1"):
        load_dotenv()
        
        # Initialize embeddings
        self.embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-mpnet-base-v2"
        )
        
        # Initialize text splitter
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len,
        )
        
        # Initialize LLM
        self.llm = TogetherLLM(
            api_key=TOGETHER_API_KEY,
            model=model_name,
            temperature=0.7,
            max_tokens=2048,
        )
        
        self.vector_store = None
        self.conversation_chain = None
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            output_key="answer",
            input_key="question",
            return_messages=True
        )

    def get_relevant_documents(self, question: str) -> Tuple[List[Document], List[float]]:
        """Get relevant documents with similarity scores."""
        if self.vector_store is None:
            return [], []
        
        # Get documents and scores from the vector store
        docs_and_scores = self.vector_store.similarity_search_with_score(
            question,
            k=2
        )
        
        # Separate documents and scores
        docs = [doc for doc, _ in docs_and_scores]
        scores = [score for _, score in docs_and_scores]
        
        return docs, scores

    def evaluate_response_quality(self, question: str, answer: str, context: str) -> float:
        """Evaluate the quality of the response based on relevance and completeness."""
        # Calculate cosine similarity between question and answer embeddings
        question_embedding = self.embeddings.embed_query(question)
        answer_embedding = self.embeddings.embed_query(answer)
        
        # Convert to numpy arrays for calculation
        q_embed = np.array(question_embedding)
        a_embed = np.array(answer_embedding)
        
        # Calculate cosine similarity
        similarity = np.dot(q_embed, a_embed) / (np.linalg.norm(q_embed) * np.linalg.norm(a_embed))
        
        return float(similarity)

    def process_pdf(self, pdf_path: str) -> str:
        """Extract text from PDF file using PyMuPDF."""
        text = ""
        with fitz.open(pdf_path) as doc:
            for page in doc:
                text += page.get_text()
        return text

    def process_documents(self, file_paths: List[str]) -> None:
        """Process multiple documents and create vector store."""
        documents = []
        
        for file_path in file_paths:
            if file_path.endswith('.pdf'):
                text = self.process_pdf(file_path)
                doc = Document(
                    page_content=text,
                    metadata={"source": file_path}
                )
                documents.append(doc)
        
        # Split documents into chunks
        splits = self.text_splitter.split_documents(documents)
        
        # Create or update vector store
        if self.vector_store is None:
            self.vector_store = FAISS.from_documents(splits, self.embeddings)
        else:
            self.vector_store.add_documents(splits)
        
        # Initialize conversation chain with enhanced retriever
        base_retriever = self.vector_store.as_retriever(
            search_kwargs={"k": 2}
        )
        
        # Add embeddings filter for better context relevance
        embeddings_filter = EmbeddingsFilter(
            embeddings=self.embeddings,
            similarity_threshold=0.7
        )
        compression_retriever = ContextualCompressionRetriever(
            base_compressor=embeddings_filter,
            base_retriever=base_retriever
        )
        
        self.conversation_chain = ConversationalRetrievalChain.from_llm(
            llm=self.llm,
            retriever=compression_retriever,
            memory=self.memory,
            return_source_documents=True,
            verbose=True,
            return_generated_question=False,
            chain_type="stuff",
            rephrase_question=False
        )

    def query(self, question: str) -> Dict:
        """Query the RAG system with a question."""
        if self.conversation_chain is None:
            raise ValueError("Please process documents before querying.")
        
        try:
            # Get relevant documents with scores
            docs, relevance_scores = self.get_relevant_documents(question)
            
            # Get the response from the conversation chain
            response = self.conversation_chain({"question": question})
            answer = response["answer"]
            
            # Evaluate response quality
            response_quality = self.evaluate_response_quality(
                question,
                answer,
                " ".join([doc.page_content for doc in docs])
            )
            
            return {
                "answer": answer,
                "source_documents": [
                    {
                        "content": doc.page_content,
                        "source": doc.metadata["source"],
                        "relevance_score": float(score)  # Convert to float for JSON serialization
                    }
                    for doc, score in zip(response["source_documents"], relevance_scores)
                ],
                "response_quality_score": response_quality,
                "average_relevance_score": float(np.mean(relevance_scores)) if relevance_scores else 0.0
            }
        except Exception as e:
            print(f"Error during query: {str(e)}")
            return {
                "answer": "Sorry, I encountered an error while processing your question.",
                "source_documents": [],
                "response_quality_score": 0.0,
                "average_relevance_score": 0.0
            }

    def save_vector_store(self, path: str) -> None:
        """Save the vector store to disk."""
        if self.vector_store is not None:
            self.vector_store.save_local(path)

    def load_vector_store(self, path: str) -> None:
        """Load the vector store from disk."""
        if os.path.exists(path):
            self.vector_store = FAISS.load_local(path, self.embeddings)
            self.conversation_chain = ConversationalRetrievalChain.from_llm(
                llm=self.llm,
                retriever=self.vector_store.as_retriever(
                    search_kwargs={"k": 2}
                ),
                memory=self.memory,
                return_source_documents=True,
            ) 

In [66]:
import os

# Initialize the RAG agent
agent = RAGAgent()

# Path to your PDF documents
pdf_directory = "/home/teng/development/projects/public/efh-hackathon/notebooks/data/Agroforestry"

pdf_files = [
    os.path.join(pdf_directory, f) 
    for f in os.listdir(pdf_directory)
    if f.endswith('.pdf')
]

# Process the documents
print("Processing documents...")
agent.process_documents(pdf_files)

# Save the vector store for future use
agent.save_vector_store("vector_store")

# Test the RAG system
questions = [
    "How do different tree intervention practices impact food security?"
]

for question in questions:
    print(f"\nQuestion: {question}")
    response = agent.query(question)
    print("\nAnswer:", response["answer"])
    print(f"Response Quality Score: {response['response_quality_score']:.2f}")
    print(f"Average Document Relevance: {response['average_relevance_score']:.2f}\n")
    print("\nSource Documents:")
    for doc in response["source_documents"]:
        print(f"  Relevance Score: {doc['relevance_score']:.2f}")
        print(f"  Source: {doc['source']}")
        print(f"  Relevant content: {doc['content'][:200]}...")

Processing documents...

Question: How do different tree intervention practices impact food security?


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

adjustment for dependence) are shown in the appendix 
(p 16).
For outcomes related to food and nutrition, the impacts 
of interventions on agricultural yields and food security 
were examined in ten studies (more than 20 different 
analyses; figure 3). The pooled effect across all 
interventions showed a significant, moderate-sized 
positive impact on agricultural yield (SMD 0·41 [95% CI 
0·11 to 0·70], p=0·0076; I²=94·9% and τ²=0·17; figure 3). 
The subgroup analysis found that interventions related 
to trees in croplands had the greatest significant effects 
(0·35 [0·04 to 0·67]; p=0·025; I²=95·2% and τ²=0·26). 
Smalle